In [5]:
#!pip install --upgrade google-cloud-vision google-cloud-storage

import os, io
import re
from google.cloud import vision
from google.cloud import storage
from google.protobuf import json_format

!pip install --upgrade python-google-places
from googleplaces import GooglePlaces, types, lang
import requests
import json

Requirement already up-to-date: python-google-places in /home/mayank/VirtualEnvs/venv/lib/python3.7/site-packages (1.4.1)


ModuleNotFoundError: No module named 'googleplaces'

In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/home/mayank/libtech/src/scripts/aap-bbmp-creds.json'

project = 'AAP-BBMP'
bucket_name = 'draft-rolls'
file_name = 'z.txt'

print(storage)
storage_client = storage.Client()
print(storage_client)
bucket = storage_client.get_bucket(bucket_name)
print(bucket)
blob = bucket.blob(file_name)
print(blob)
print(blob.upload_from_filename(file_name))

<module 'google.cloud.storage' from '/home/mayank/anaconda3/lib/python3.7/site-packages/google/cloud/storage/__init__.py'>


<Bucket: draft-rolls>
<Blob: draft-rolls, z.txt, None>


None


In [ ]:
# coding: utf-8

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/home/mayank/libtech/src/scripts/aap-bbmp-creds.json'
client = vision.ImageAnnotatorClient()

batch_size = 100
mime_type = 'application/pdf'
feature = vision.types.Feature(
    type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

gcs_source_uri = 'gs://draft-rolls/bbmp-wards-2020.pdf'
gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
input_config = vision.types.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

gcs_destination_uri = 'gs://draft-rolls/bbmp-wards-2020.pdf_'
gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
output_config = vision.types.OutputConfig(gcs_destination=gcs_destination, batch_size=batch_size)

async_request = vision.types.AsyncAnnotateFileRequest(
    features=[feature], input_config=input_config, output_config=output_config)

operation = client.async_batch_annotate_files(requests=[async_request])
operation.result(timeout=180)

storage_client = storage.Client()
match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
bucket_name = match.group(1)
prefix = match.group(2)
bucket = storage_client.get_bucket(bucket_name)

# List object with the given prefix
blob_list = list(bucket.list_blobs(prefix=prefix))
print('Output files:')
for blob in blob_list:
    print(blob.name)

output = blob_list[0]
print(type(output))
print(output)
json_string = output.download_as_string()
response = json_format.Parse(
            json_string, vision.types.AnnotateFileResponse())

first_page_response = response.responses[3]
annotation = first_page_response.full_text_annotation

print(u'Full text:')
print(annotation.text)


In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'image_to_text_googleVision.json'
client = vision.ImageAnnotatorClient()


In [ ]:
batch_size = 40
mime_type = 'application/pdf'
feature = vision.types.Feature(
    type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

In [ ]:
gcs_source_uri = 'gs://scanned_pdfs/Mynk.pdf'
gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
input_config = vision.types.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

In [ ]:
gcs_destination_uri = 'gs://scanned_pdfs/pdf_result'
gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
output_config = vision.types.OutputConfig(gcs_destination=gcs_destination, batch_size=batch_size)

In [ ]:
async_request = vision.types.AsyncAnnotateFileRequest(
    features=[feature], input_config=input_config, output_config=output_config)

In [ ]:
operation = client.async_batch_annotate_files(requests=[async_request])
operation.result(timeout=180)

storage_client = storage.Client()
match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
bucket_name = match.group(1)
prefix = match.group(2)
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
# List object with the given prefix
blob_list = list(bucket.list_blobs(prefix=prefix))
print('Output files:')
for blob in blob_list:
    print(blob.name)

output = blob_list[0]
json_string = output.download_as_string()
response = json_format.Parse(
            json_string, vision.types.AnnotateFileResponse())

first_page_response = response.responses[3]
annotation = first_page_response.full_text_annotation

print(u'Full text:')
print(annotation.text)